In [4]:
import numpy as np
from music21 import converter, note, duration
import pandas as pd

# Custom Midi note object
from midi_note import MIDINote

#utility
import random
import copy
from bidict import bidict

## Preprocessing Simulation

This notebook is a gist of the overall preprocessing step the researcher applied to the dataset.

![](http://www.musicxlab.com/img/logo-fine.4b1b5226.png)
The study used the dataset from [Music-X-Lab](http://www.musicxlab.com/) from the study [POP909: A Pop-song Dataset for Music Arrangement Generation](https://arxiv.org/abs/2008.07142).

Clone the data repository at [POP909](https://github.com/music-x-lab/POP909-Dataset).

In [5]:
# path of the data storage
PATH = "POP909"

####  Data Selection for simulation
The researcher collected all the songs belonging to 4 musical key for the simulation from the dataset:
 - E minor
 - D minor
 - A major
 - C Major

In [6]:
songs_by_key = dict()

songs_by_key["Emin"] = ['033', '034', '056', '071', '078', '116', '130', '240', '272', '276', '283', '317', '322',
                        '380', '457', '458', '476', '501', '520', '560', '566', '578', '625', '653', '667', '680',
                        '693', '740', '772', '788', '854', '899']
songs_by_key["Dmin"] = ['027', '040', '073', '103', '125', '141', '151', '158', '198', '226', '284', '295', '297',
                        '344', '378', '384', '389', '446', '451', '587', '604', '610', '627', '682', '689', '791',
                        '842', '845', '852', '859', '894']
songs_by_key["Amaj"] = ['058', '086', '102', '155', '174', '183', '208', '231', '291', '346', '353', '362', '410',
                        '463', '475', '478', '497', '499', '615', '641', '687', '691', '721', '728', '751', '774',
                        '810', '867', '891']
songs_by_key["Cmaj"] = ['038', '055', '068', '079', '131', '132', '136', '171', '172', '185', '203', '211', '216',
                        '233', '243', '278', '293', '312', '319', '320', '326', '331', '368', '386', '432', '459',
                        '493', '496', '548', '570', '591', '603', '612', '621', '702', '710', '714', '722', '735',
                        '761', '793', '824', '833', '873', '888', '892', '909']

#### Song Selection
Set how many songs will be selected for each music key using the *sample_limit* parameter.
</br>
Note: *To simulate using random sample, set* **random_selection=<span style="color:green">True </span>**

In [7]:
sample_limit = 4
all_song_ids = []
random_selection = False
for key, song_ids in songs_by_key.items():
    if random_selection:
        all_song_ids += random.sample(song_ids, sample_limit)
    else:
        all_song_ids += songs_by_key[key][:sample_limit]

### Parse Raw Notes Function

This function parses a midi file into a python runnable data structure, filters the melody part of the midi song, and save the properties (duration type, length, pitch) a note to create a Note Object.
<p> Example of a Note object: { note: C#4, duration_type: whole, length: 4 }

In [8]:
def parse_raw_notes(file_path):
    print("Loading Music File:", file_path)
    raw_notes = []
    midi_data = converter.parse(file_path)
    for part in midi_data.parts:
        if part.partName == 'MELODY':
            midi_elements = part.recurse()
            for element in midi_elements:
                if isinstance(element, note.Note):
                    note_duration = duration.Duration()
                    note_duration.quarterLength = element.quarterLength
                    raw_note = str(element.pitch)
                    raw_notes.append(MIDINote(raw_note, str(note_duration.type), str(element.quarterLength)).as_map)
    return raw_notes


parsed_midi_notes = []
for song_id in all_song_ids:
    song_path = f"{PATH}/{song_id}/{song_id}.mid"
    parsed_raw_notes = parse_raw_notes(song_path)
    parsed_midi_notes.append(parsed_raw_notes)

Loading Music File: POP909/033/033.mid
Loading Music File: POP909/034/034.mid
Loading Music File: POP909/056/056.mid
Loading Music File: POP909/071/071.mid
Loading Music File: POP909/027/027.mid
Loading Music File: POP909/040/040.mid
Loading Music File: POP909/073/073.mid
Loading Music File: POP909/103/103.mid
Loading Music File: POP909/058/058.mid
Loading Music File: POP909/086/086.mid
Loading Music File: POP909/102/102.mid
Loading Music File: POP909/155/155.mid
Loading Music File: POP909/038/038.mid
Loading Music File: POP909/055/055.mid
Loading Music File: POP909/068/068.mid
Loading Music File: POP909/079/079.mid


### This function returns a random song on a given list
This function will be used throughout the simulation to randomly select and display a song after every preprocessing steps.

In [9]:
def get_random_song(song_list):
    rand_song_id = random.randint(0, len(song_list) - 1)
    rand_song = song_list[rand_song_id]
    return rand_song, rand_song_id

### View random song
Get a general view of what a parsed midi song looks like.

In [10]:
random_song, random_song_id = get_random_song(parsed_midi_notes)
data_frame = pd.DataFrame(random_song)
print("Randomly selected song id:", random_song_id)
data_frame.head(10)

Randomly selected song id: 10


,note,duration_type,length
0,C#5,quarter,2/3
1,C#5,32nd,1/12
2,E5,quarter,2/3
3,E5,32nd,1/12
4,F#5,quarter,2/3
5,F#5,32nd,1/12
6,A4,quarter,2/3
7,A4,32nd,1/12
8,G#4,quarter,2/3
9,G#4,32nd,1/12


### View random song with flats
Randomly select song to check/show flat notes

In [14]:
# if default set id = 13, 5
random_song, random_song_id = get_random_song(parsed_midi_notes)
data_frame = pd.DataFrame(random_song)

flats = data_frame[data_frame["note"].str.contains("-")]
print("Randomly selected song id:", random_song_id)
flats.head(10)

Randomly selected song id: 13


,note,duration_type,length
386,E-5,eighth,1/3
387,E-5,quarter,2/3


## Initial preprocessing
The initial preprocessing includes mapping of **flat** notes to its corresponding **sharp** values

Example: *E-5 (E flat, 5th Octave) is equal to D#5 (D sharp, 5th Octave)*

In [22]:
flats_map = {
    'D-': 'C#',
    'E-': 'D#',
    'G-': 'F#',
    'A-': 'G#',
    'B-': 'A#'
}


def map_flat(song):
    for song_note in song:
        if "-" in song_note["note"]:
            flat_note = song_note["note"][:2]
            song_note["note"] = song_note["note"].replace(flat_note, flats_map[flat_note])


def initial_preprocess(songs):
    for song in songs:
        map_flat(song)
    return songs


initial_preprocessed_notes = initial_preprocess(copy.deepcopy(parsed_midi_notes))

### View random song to check for flats
All flats are mapped to sharps, therefore, selecting songs that contain flat values will show empty tables.

In [ ]:
random_song, random_song_id = get_random_song(initial_preprocessed_notes)
data_frame = pd.DataFrame(random_song)
print("Randomly selected song id:", random_song_id)
# show if flats still exists
flats = data_frame[data_frame["note"].str.contains("-")]
flats.head(10)

### View random song to check for complex notes
Some songs contain complex notes, they are notes that contain *advanced duration* (notes that are complex and tuplets with fractional durations not found on basic music notes

In [70]:
random_song, random_song_id = get_random_song(initial_preprocessed_notes)
data_frame = pd.DataFrame(random_song)
# show complex notes
flats = data_frame[data_frame["duration_type"] == "complex"]
print("Randomly selected song id:", random_song_id)
flats.head(10)

Randomly selected song id: 5


,note,duration_type,length


### View random song to check for 32nd duration typed notes
Some songs contain notes that have 32nd duration which is not included in our proposed output classes.

In [ ]:
random_song, random_song_id = get_random_song(initial_preprocessed_notes)
data_frame = pd.DataFrame(random_song)
# show 32nd notes
flats = data_frame[data_frame["duration_type"] == "32nd"]
print("Randomly selected song id:", random_song_id)
flats.head(10)

## Second Preprocessing
The second preprocessing involves mapping the standard length for each duration type, mapping complex notes
to its nearest non-complex note, and rounding up of all 32nd duration-typed notes to 16th .

In [72]:
duration_map = {
    'whole': 4.0,
    'half': 2.0,
    'quarter': 1.0,
    'eighth': 0.5,
    '16th': 0.25,
}


def map_duration(song):
    duration_type = song["duration_type"]
    if duration_type in duration_map:
        song["length"] = duration_map[duration_type]
        return
    # if note is 32nd then it is transformed in to 16th
    if duration_type == "32nd":
        song["duration_type"] = "16th"
        song["length"] = 0.25

    # if a note is complex then map it to its nearest non-complex note
    if duration_type == "complex":
        length = float(song["length"])
        if length <= 0.25:
            song["duration_type"] = "16th"
            song["length"] = "0.25"

        minimum_duration = "whole"
        distance = float('inf')
        for duration_type, duration_length in duration_map.items():
            if abs(duration_length - length) < distance:
                distance = abs(duration_length - length)
                minimum_duration = duration_type
        song["duration_type"] = minimum_duration
        song["length"] = duration_map[minimum_duration]


def second_preprocess(songs):
    for song in songs:
        for song_note in song:
            map_duration(song_note)
    return songs


second_preprocess_data = second_preprocess(copy.deepcopy(initial_preprocessed_notes))


#### Check for complex notes are still present

In [75]:
random_song, random_song_id = get_random_song(second_preprocess_data)
data_frame = pd.DataFrame(random_song)
# show complex notes
flats = data_frame[data_frame["duration_type"] == "complex"]
print("Randomly selected song id:", random_song_id)
flats.head(10)

Randomly selected song id: 1


,note,duration_type,length


#### Check if 32nd notes are still present

In [76]:
random_song, random_song_id = get_random_song(second_preprocess_data)
data_frame = pd.DataFrame(random_song)
# show 32nd notes
flats = data_frame[data_frame["duration_type"] == "32nd"]
print("Randomly selected song id:", random_song_id)
flats.head(10)

Randomly selected song id: 1


,note,duration_type,length


### Utility functions in checking octave frequency

#### Function that returns octave frequency in all songs

In [82]:
def get_octave_frequency(songs):
    octave_freq = {}
    for song in songs:
        for song_note in song:
            octave = song_note["note"][-1]
            if octave not in octave_freq:
                octave_freq[octave] = 0
            octave_freq[octave] += 1
    return octave_freq


octave_frequency = get_octave_frequency(second_preprocess_data)
octave_frequency

{'4': 2582, '5': 3585, '6': 1007, '3': 41}

#### Function that returns octave frequency of a song

In [83]:
def octave_frequency_by_song(song):
    octave_freq = {}
    for song_note in song:
        octave = song_note["note"][-1]
        if octave not in octave_freq:
            octave_freq[octave] = 0
        octave_freq[octave] += 1
    return octave_freq

#### Function that returns octave frequency of each song

In [89]:
def octave_frequency_by_songs(songs):
    octave_freq_list = []
    for song in songs:
        octave_freq_by_song = octave_frequency_by_song(song)
        octave_freq_list.append(octave_freq_by_song)
    return octave_freq_list


#individual songs
octave_frequency_list = octave_frequency_by_songs(second_preprocess_data)
for i, ofl in enumerate(octave_frequency_list):
    print(f"Song no: {i}")
    print(ofl)


Song no: 0
{'4': 223, '5': 151}
Song no: 1
{'5': 369, '6': 108, '4': 3}
Song no: 2
{'4': 302, '5': 163}
Song no: 3
{'4': 307, '5': 173, '6': 1}
Song no: 4
{'5': 153, '6': 231}
Song no: 5
{'5': 256, '6': 148}
Song no: 6
{'4': 431, '3': 32, '5': 22}
Song no: 7
{'5': 272, '6': 199}
Song no: 8
{'5': 174, '4': 207}
Song no: 9
{'5': 237, '4': 180}
Song no: 10
{'5': 328, '4': 154, '6': 15}
Song no: 11
{'4': 228, '5': 261}
Song no: 12
{'5': 370, '4': 72, '6': 12}
Song no: 13
{'4': 308, '3': 9, '5': 179, '6': 2}
Song no: 14
{'4': 164, '5': 267}
Song no: 15
{'5': 210, '6': 291, '4': 3}


### View random song containing notes not belonging to the dominant octaves
Some songs contain notes not belonging to 4th and fifth octaves (outlier notes)

In [103]:
random_song, random_song_id = get_random_song(second_preprocess_data)
data_frame = pd.DataFrame(random_song)

flats = data_frame[(~data_frame["note"].str.contains("4")) & (~data_frame["note"].str.contains("5"))]
print("Randomly selected song id:", random_song_id)
flats.head(10)


Randomly selected song id: 1


,note,duration_type,length
28,C6,16th,0.25
29,C6,16th,0.25
30,C6,16th,0.25
65,C6,16th,0.25
66,C6,16th,0.25
67,C6,16th,0.25
110,E6,eighth,0.50
112,D6,16th,0.25
113,E6,16th,0.25
114,E6,eighth,0.50


## Third Preprocessing


#### This function gets the frequency of octaves of a song and sorts them
The sorted frequency of octaves will identify the most dominant octaves of the song.

In [111]:
def octave_preprocessing(song):
    octave_freq = octave_frequency_by_song(song)

    if len(octave_freq) >= 2 or "4" not in octave_freq or "5" not in octave_freq:
        octave_sorted = []
        for song_key, song_key_frequency in octave_freq.items():
            octave_sorted.append((song_key, song_key_frequency))
        octave_sorted.sort(key=lambda x: -x[1])
        print(octave_sorted)
        to_dual_octaves(song, octave_sorted)

#### This function rescales octaves of notes to the dominant octaves of the song
There are only two octaves proposed in the output class of the model. Hence, songs with
multiple octaves should rescale their outlier octaves to the nearest dominant octaves.

In [112]:
def to_dual_octaves(song, octave_sorted):
    lower_octave_details, higher_octave_details = sorted(octave_sorted[:2], key=lambda x: x[0])
    for song_note in song:
        octave = song_note["note"][-1]
        if octave < lower_octave_details[0]:
            song_note["note"] = song_note["note"][:-1] + lower_octave_details[0]
        elif octave > higher_octave_details[0]:
            song_note["note"] = song_note["note"][:-1] + higher_octave_details[0]

    octave_freq = octave_frequency_by_song(song)

    if "4" not in octave_freq or "5" not in octave_freq:
        rescale_octave(song, lower_octave_details, higher_octave_details)


#### This function rescales octaves to fit the octaves in output class
Some songs have dominant octaves from 3rd to 4th octaves or from 5th to 6th octaves. This method rescales dominant
octaves of the song to 4th and 5th octaves to fit the proposed output classes.

In [113]:
def rescale_octave(song, lower_octave_details, higher_octave_details):
    lower_octave = lower_octave_details[0]
    higher_octave = higher_octave_details[0]
    for midi_note in song:
        octave = midi_note["note"][-1]
        if octave == lower_octave:
            midi_note["note"] = midi_note["note"][:-1] + "4"
        elif octave == higher_octave:
            midi_note["note"] = midi_note["note"][:-1] + "5"
    note_freq_per_note = octave_frequency_by_song(song)
    print(note_freq_per_note)

In [114]:
# run third preprocessing
def third_preprocessing(songs):
    for song in songs:
        octave_preprocessing(song)
    return songs


third_preprocessed_data = third_preprocessing(copy.deepcopy(second_preprocess_data))

[('4', 223), ('5', 151)]
[('5', 369), ('6', 108), ('4', 3)]
{'4': 372, '5': 108}
[('4', 302), ('5', 163)]
[('4', 307), ('5', 173), ('6', 1)]
[('6', 231), ('5', 153)]
{'4': 153, '5': 231}
[('5', 256), ('6', 148)]
{'4': 256, '5': 148}
[('4', 431), ('3', 32), ('5', 22)]
{'5': 453, '4': 32}
[('5', 272), ('6', 199)]
{'4': 272, '5': 199}
[('4', 207), ('5', 174)]
[('5', 237), ('4', 180)]
[('5', 328), ('4', 154), ('6', 15)]
[('5', 261), ('4', 228)]
[('5', 370), ('4', 72), ('6', 12)]
[('4', 308), ('5', 179), ('3', 9), ('6', 2)]
[('5', 267), ('4', 164)]
[('6', 291), ('5', 210), ('4', 3)]
{'4': 213, '5': 291}


### View random song containing notes not belonging to the dominant octaves
Randomly select a song to check the effect of third preprocessing

In [115]:
random_song, random_song_id = get_random_song(third_preprocessed_data)
data_frame = pd.DataFrame(random_song)
# show outlier octaves notes
flats = data_frame[(~data_frame["note"].str.contains("4")) & (~data_frame["note"].str.contains("5"))]
print("Randomly selected song id:", random_song_id)
flats.head(10)

Randomly selected song id: 2


,note,duration_type,length


The result of the third preprocessing based on octave_frequency on the sample dataset

In [117]:
octave_frequency = get_octave_frequency(third_preprocessed_data)
octave_frequency

{'4': 3452, '5': 3763}

In [312]:
"""
TO TRAINABLE DATA
"""

'\nTO TRAINABLE DATA\n'

In [313]:

notes_map = bidict({
    'C': 0,
    'C#': 5,
    'D': 10,
    'D#': 15,
    'E': 20,
    'F': 25,
    'F#': 30,
    'G': 35,
    'G#': 40,
    'A': 45,
    'A#': 50,
    'B': 55,
})
train_duration_map = bidict({
    'whole': 0,
    'half': 1,
    'quarter': 2,
    'eighth': 3,
    '16th': 4,
})


def map_note_to_int(song_note):
    pitch = song_note["note"][:-1]
    octave = int(song_note["note"][-1])

    return notes_map[pitch] + train_duration_map[song_note["duration_type"]] + ((octave - 4) * 60)

In [314]:
def song_map_to_int(song):
    song_notes_to_int = []
    for song_note in song:
        song_notes_to_int.append(map_note_to_int(song_note))
    return song_notes_to_int


def songs_map_to_int(songs):
    songs_in_int = []
    for song in songs:
        songs_in_int.append(song_map_to_int(song))
    return songs_in_int

In [320]:
sequence_length = 50


def shift_append(song_in_int, seq_len):
    _X = []
    _Y = []
    limit = len(song_in_int) - seq_len
    for index in range(limit):
        _X.append(song_in_int[index:index + seq_len])
        _Y.append(song_in_int[index + seq_len])

    return _X, _Y


def shift_append_songs(songs_in_int, seq_len):
    X = []
    Y = []
    for song_in_int in songs_in_int:
        x, y = shift_append(song_in_int, seq_len)
        X += x
        Y += y
    return np.array(X), np.array(Y)

In [321]:
songs_map_int = songs_map_to_int(third_preprocessed_data)
len(songs_map_int)

16

In [322]:
X_train, Y_train = shift_append_songs(songs_map_int, sequence_length)

In [324]:
X_train.shape

(6415, 50)

In [325]:
Y_train.shape

(6415,)

In [326]:
with open(f'data/x_train_sample.npy', 'wb') as f:
    np.save(f, X_train)
with open(f'data/y_train_sample.npy', 'wb') as f:
    np.save(f, Y_train)

